# 20 뉴스그룹 분류

In [1]:
import numpy as np
import pandas as pd

In [2]:
##from sklearn.datasets import load 로 내장데이터 가지고올수 있다
from sklearn.datasets import fetch_20newsgroups #fetch는 다운받아서 온다
news =  fetch_20newsgroups(subset='all', random_state=2021)   ##옛날 게시판

## 데이터 탐색

In [3]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
pd.Series(news.target).value_counts().sort_index() #dict 형태여서 이렇게

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [7]:
train_news = fetch_20newsgroups(
    subset='train', random_state=2021, 
    remove = ('headers', 'footers', 'quotes') #본문만 필요해 지운다
)
len(train_news.data)

11314

In [8]:
test_news = fetch_20newsgroups(
    subset='test', random_state=2021, 
    remove = ('headers', 'footers', 'quotes') #본문만 필요해 지운다
)
len(test_news.data)

7532

## 텍스트 데이터에 대해서 전처리

In [35]:
train_df = pd.DataFrame({
    'article' : train_news.data
})
test_df = pd.DataFrame({
    'article' : test_news.data
})
train_news.data[1]

']Is it possible to do a "wheelie" on a motorcycle with shaft-drive?\n\nyes.\n'

### train dataset

In [36]:
# 특수문자 제거
train_df.article = train_df.article.str.replace("[^a-zA-Z]",' ')
train_df.article[1]

<ipython-input-36-666d7d12a092>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df.article = train_df.article.str.replace("[^a-zA-Z]",' ')


' Is it possible to do a  wheelie  on a motorcycle with shaft drive   yes  '

In [37]:
# 길이가 3이하인 단어 제거
import re
train_df.article = train_df.article.apply(lambda x : ' '.join([w for w in x.split() if len(w)> 3]))

In [38]:
# 소문자로 변환
train_df.article = train_df.article.apply(lambda x : x.lower())

In [39]:
# 길이가 3이하인 단어 제거
import re
train_df.article = train_df.article.apply(lambda x : ' '.join([w.lower() for w in x.split() if len(w)> 3]))

In [40]:
train_df.article[1]

'possible wheelie motorcycle with shaft drive'

### 테스트 데이터셋

In [41]:
test_df.article = test_df.article.str.replace("[^a-zA-Z]",' ')
import re
test_df.article = test_df.article.apply(lambda x : ' '.join([w.lower() for w in x.split() if len(w)> 3]))

<ipython-input-41-1891009eebed>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df.article = test_df.article.str.replace("[^a-zA-Z]",' ')


## 텍스트 변환

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
tvect = TfidfVectorizer(stop_words='english')
tvect.fit(train_df.article)

TfidfVectorizer(stop_words='english')

In [44]:
X_train = tvect.transform(train_df.article)
X_test = tvect.transform(test_df.article)

In [45]:
X_train.shape, X_test.shape

((11314, 64133), (7532, 64133))

In [30]:
y_train = train_news.target
y_test = test_news.target

### 훈련 예측 평가

In [21]:
# support vect machine 의 classifier 사용
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [46]:
pred = svc.predict(X_test)

In [47]:
accuracy_score(y_test, pred)

0.6488316516197558